<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_01/6%20-%20C2_W1_Lab_1_cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Uso de imagens mais sofisticadas com redes neurais convolucionais

Você viu na seção anterior como usar uma CNN para tornar mais eficiente o reconhecimento de imagens de cavalos e humanos geradas por computador.

Nesta lição, você levará isso para o próximo nível: criar um modelo para classificar imagens reais de gatos e cachorros.

Assim como o conjunto de dados de cavalos e humanos, as imagens do mundo real também têm diferentes formas, proporções, etc., e você precisará levar isso em consideração ao preparar os dados.

Neste laboratório, você primeiro reverá como criar CNNs, preparará os dados com o `ImageDataGenerator` e examinará os resultados. Você seguirá estas etapas:

1.   Explore os dados de exemplo de `Dogs vs. Cats`
2.   Crie e treine uma rede neural para classificar os dois animais de estimação
3.   Avalie a acurácia do treinamento e da validação

Você usará os resultados obtidos aqui nos próximos laboratórios para aprimorá-los, principalmente para evitar o overfitting. Vamos começar!

## Baixar e inspecionar o conjunto de dados

Você começará fazendo o download do conjunto de dados.

Trata-se de um `.zip` com 2.000 imagens JPG de cães e gatos.

É um subconjunto do conjunto de dados ["Dogs vs. Cats"] (https://www.kaggle.com/c/dogs-vs-cats/data) disponível no Kaggle, que contém 25.000 imagens.

Você usará apenas 2.000 do conjunto de dados completo para reduzir o tempo de treinamento para fins educacionais.

In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

Em seguida, você o extrairá para o diretório atual.

In [ ]:
import zipfile

# Descompacte o arquivo
local_zip = './cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

O conteúdo do .zip é extraído para o diretório base `./cats_and_dogs_filtered`, que contém os subdiretórios `train` e `validation` para os conjuntos de dados de treinamento e validação (você pode ignorar o `vectorize.py` na saída da próxima célula). 

Lembre-se
* **conjunto de treinamento** são os dados usados para informar ao modelo de rede neural que "esta é a aparência de um gato" e "esta é a aparência de um cachorro".
*  **conjunto de validação** são imagens de gatos e cachorros que a rede neural não verá como parte do treinamento. Você pode usá-lo para testar se ele se sai bem ou mal ao avaliar se uma imagem contém um gato ou um cachorro. (Consulte o [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/validation/check-your-intuition) se quiser relembrar os conjuntos de treinamento, validação e teste).

Esses subdiretórios, por sua vez, contêm subdiretórios `cats` e `dogs`.

In [ ]:
import os

base_dir = 'cats_and_dogs_filtered'

print("Conteúdo do diretório base:")
print(os.listdir(base_dir))

print("\nConteúdo do diretório de treino:")
print(os.listdir(f'{base_dir}/train'))

print("\nConteúdo do diretório validação:")
print(os.listdir(f'{base_dir}/validation'))

Você pode atribuir cada um desses diretórios a uma variável para poder usá-la posteriormente.

In [ ]:
import os

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Diretório com imagens de gatos/cães para validação
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


Agora veja como são os nomes dos arquivos nos diretórios `cats` e `dogs` `train` (as convenções de nomenclatura de arquivos são as mesmas no diretório `validation`):

In [ ]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

Vamos descobrir o número total de imagens de cães e gatos nos diretórios `train` e `validation`:

In [ ]:
print('total de imagens de gatos para treinamento :', len(os.listdir(      train_cats_dir ) ))
print('total de imagens de cães para treinamento :', len(os.listdir(      train_dogs_dir ) ))

print('total de imagens de gatos para validação :', len(os.listdir( validation_cats_dir ) ))
print('total de imagens de cães para validação :', len(os.listdir( validation_dogs_dir ) ))

Para cães e gatos, você tem 1.000 imagens de treinamento e 500 imagens de validação.

Agora, dê uma olhada em algumas imagens para ter uma noção melhor de como são os conjuntos de dados de cães e gatos. Primeiro, configure os parâmetros do `matplotlib`:

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parâmetros para nosso gráfico; produziremos imagens em uma configuração 4x4
nrows = 4
ncols = 4

pic_index = 0 # Índice para iteração de imagens

Agora, exiba um lote de 8 imagens de gatos e 8 de cachorros. Você pode executar a célula novamente para ver um novo lote a cada vez:

In [ ]:
# Configure o matplotlib fig e dimensione-o para caber em fotos 4x4
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Configure o subplot; os índices do subplot começam em 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Não mostre os eixos (ou linhas de grade)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


Talvez não seja óbvio observar as imagens nessa grade, mas uma observação importante é que essas imagens têm todas as formas e tamanhos (assim como o conjunto de dados "cavalos ou humanos").

Portanto, antes de treinar uma rede neural com elas, você precisará ajustar as imagens. Você verá isso nas próximas seções.

## Criando um modelo pequeno a partir do zero para chegar a ~72% de acurácia

Para treinar uma rede neural para lidar com as imagens, você precisará que elas tenham um tamanho uniforme. Você escolherá 150x150 pixels para isso, e verá o código que pré-processa as imagens para esse formato em breve. 

Você pode definir o modelo importando o Tensorflow e usando a API do Keras. Aqui está o código completo primeiro e a discussão vem depois. Isso é muito semelhante aos modelos que você criou no Curso 1.


In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Observe que a forma de entrada é o tamanho desejado da imagem 150x150 com 3 bytes de cor
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Achatar os resultados para alimentar um DNN
    tf.keras.layers.Flatten(), 
    # Camada oculta de 512 neurônios
    tf.keras.layers.Dense(512, activation='relu'), 
    # Apenas 1 neurônio de saída. Ele conterá um valor de 0 a 1, sendo 0 para uma classe ("gatos") e 1 para a outra ("cães")
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

Você definiu uma camada `Sequential` como antes, adicionando algumas camadas convolucionais primeiro.

Observe o parâmetro `input_shape` desta vez. Aqui é onde você coloca o tamanho `150x150` e `3` para a profundidade de cor, pois você tem imagens coloridas.

Em seguida, você adiciona algumas camadas convolucionais e achata o resultado final para alimentar as camadas densamente conectadas.

Observe que, como você está enfrentando um problema de classificação de duas classes, ou seja, um *problema de classificação binária*, você terminará a rede com uma ativação [*sigmoid*](https://wikipedia.org/wiki/Sigmoid_function).

A saída da rede será um único escalar entre `0` e `1`, codificando a probabilidade de que a imagem atual seja da classe `1` (em oposição à classe `0`).

Você pode analisar a arquitetura da rede com o método `model.summary()`: 

In [ ]:
model.summary()

A coluna `output_shape` mostra como o tamanho do seu mapa de recursos evolui em cada camada sucessiva.

A operação de convolução remove os pixels mais externos das dimensões originais, e cada camada de agrupamento os reduz pela metade.

Em seguida, você configurará as especificações para o treinamento do modelo.

Você treinará nosso modelo com a perda `binary_crossentropy`, porque se trata de um problema de classificação binária e sua ativação final é um sigmoide.

Usaremos o otimizador `rmsprop` com uma taxa de aprendizado de `0,001`.

Durante o treinamento, você deverá monitorar a acurácia da classificação.

**NOTA**: Nesse caso, o uso do [algoritmo de otimização RMSprop](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) é preferível ao [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), porque o RMSprop automatiza o ajuste da taxa de aprendizado para nós. (Outros otimizadores, como [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) e [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), também adaptam automaticamente a taxa de aprendizado durante o treinamento e funcionariam igualmente bem aqui).

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

### Pré-processamento de dados

A próxima etapa é configurar os geradores de dados que lerão as imagens nas pastas de origem, convertê-las em tensores `float32` e alimentá-las (com seus rótulos) ao modelo. Você terá um gerador para as imagens de treinamento e outro para as imagens de validação. Esses geradores produzirão lotes de imagens de tamanho 150x150 e seus rótulos (binários).

Como você já deve saber, os dados que entram nas redes neurais geralmente devem ser normalizados de alguma forma para torná-los mais fáceis de serem processados pela rede (ou seja, não é comum alimentar uma ConvNet com pixels brutos). Nesse caso, você pré-processará as imagens normalizando os valores de pixel para que fiquem no intervalo `[0, 1]` (originalmente todos os valores estão no intervalo `[0, 255]`).

No Keras, isso pode ser feito por meio da classe `keras.preprocessing.image.ImageDataGenerator` usando o parâmetro `rescale`. Essa classe `ImageDataGenerator` permite que você instancie geradores de lotes de imagens aumentadas (e seus rótulos) por meio de `.flow(data, labels)` ou `.flow_from_directory(directory)`.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas as imagens serão redimensionadas em 1,/255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Imagens de treinamento de fluxo em lotes de 20 usando o gerador train_datagen
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Imagens de validação de fluxo em lotes de 20 usando o gerador test_datagen
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))


### Treinamento
Agora você treinará em todas as 2.000 imagens disponíveis, por 15 épocas, e monitorará a acurácia também nas 1.000 imagens do conjunto de validação.

Observe os valores por época.

Você verá 4 valores por época: perda, acurácia, perda de validação e acurácia de validação. 

A "perda" e a "acurácia" são ótimos indicadores do progresso no treinamento. A `perda` mede a previsão do modelo atual em relação aos rótulos conhecidos, calculando o resultado. A "acurácia", por outro lado, é a porção de suposições corretas. 


In [ ]:
history = model.fit(
            train_generator,
            epochs=15,
            validation_data=validation_generator,
            verbose=2
            )

### Previsão do modelo

Agora, dê uma olhada na execução de uma previsão usando o modelo. Esse código permitirá que você escolha um ou mais arquivos do seu sistema de arquivos, carregue-os e execute-os por meio do modelo, fornecendo uma indicação de que o objeto é um gato ou um cachorro.

Observação:** Versões antigas do navegador Safari podem ter problemas de compatibilidade com o bloco de código abaixo. Se você receber um erro após selecionar as imagens a serem carregadas, considere a possibilidade de atualizar seu navegador para a versão mais recente. Se não for possível, comente ou ignore o bloco de código abaixo, descomente o próximo bloco de código e execute-o._

In [ ]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded=files.upload()

for fn in uploaded.keys():
 
  # Previsão de imagens
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))
  
  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0.5:
    print(fn + " é um cachorro")
  else:
    print(fn + " é um gato")
 

### Visualizando representações intermediárias

Para ter uma ideia do tipo de recursos que sua CNN aprendeu, uma coisa divertida a se fazer é visualizar como uma entrada é transformada à medida que passa pelo modelo.

Você pode escolher uma imagem aleatória do conjunto de treinamento e, em seguida, gerar uma figura em que cada linha é a saída de uma camada e cada imagem na linha é um filtro específico nesse mapa de recursos de saída. Execute novamente essa célula para gerar representações intermediárias para uma variedade de imagens de treinamento.

In [ ]:
import numpy as np
import random
from tensorflow.keras.utils import img_to_array, load_img

# Definir um novo modelo que receberá uma imagem como entrada e produzirá
# representações intermediárias para todas as camadas do modelo anterior
successive_outputs = [layer.output for layer in model.layers]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepare uma imagem de entrada aleatória do conjunto de treinamento.
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]
img_path = random.choice(cat_img_files + dog_img_files)
img = load_img(img_path, target_size=(150, 150))  # esta é uma imagem PIL
x   = img_to_array(img)                           # Matriz Numpy com formato (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Matriz Numpy com formato (1, 150, 150, 3)

# Escala de 1/255
x /= 255.0

# Execute a imagem na rede, obtendo assim todas as
# representações intermediárias para essa imagem.
successive_feature_maps = visualization_model.predict(x)

# Esses são os nomes das camadas, para que você possa tê-las como parte de nosso gráfico
layer_names = [layer.name for layer in model.layers]

# Exibir as representações
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Faça isso apenas para as camadas conv / maxpool, não para as camadas totalmente conectadas
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # Número de recursos no mapa de recursos
    size       = feature_map.shape[ 1]  # Forma do mapa de recursos (1, tamanho, tamanho, n_recursos)
    
    # Colocar as imagens em mosaico nessa matriz
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Pós-processar o recurso para que fique visualmente agradável
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Coloque cada filtro em uma grade horizontal

    #-----------------
    # Exibir a grade
    #-----------------
    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 

Você pode ver acima como os pixels destacados se transformam em representações cada vez mais abstratas e compactas, especialmente na grade inferior. 

As representações a jusante começam a destacar aquilo a que a rede presta atenção e mostram cada vez menos recursos sendo "ativados"; a maioria é definida como zero. Isso é chamado de _esparsidade de representação_ e é um recurso fundamental da aprendizagem profunda. Essas representações carregam cada vez menos informações sobre os pixels originais da imagem, mas informações cada vez mais refinadas sobre a classe da imagem. Você pode pensar em uma convnet (ou em uma rede profunda em geral) como um pipeline de destilação de informações em que cada camada filtra os recursos mais úteis.

### Avaliando a acurácia e a perda do modelo

Você traçará a acurácia e a perda do treinamento/validação conforme coletadas durante o treinamento:

In [ ]:
#-----------------------------------------------------------
# Recupere uma lista de resultados de lista em dados de treinamento e teste
# conjuntos para cada época de treinamento
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Obter o número de épocas

#------------------------------------------------
# Plotar a acurácia do treinamento e da validação por época
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Acurácia de treino e validação')
plt.figure()

#------------------------------------------------
# Plotar a perda de treinamento e validação por época
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Perda de treino e validação'   )

Como você pode ver, o modelo está **sobreajustando** como se estivesse saindo de moda. A acurácia do treinamento (em azul) se aproxima de 100%, enquanto a acurácia da validação (em laranja) fica em 70%. A perda de validação atinge seu mínimo após apenas cinco épocas.

Como você tem um número relativamente pequeno de exemplos de treinamento (2000), o ajuste excessivo deve ser a principal preocupação. O ajuste excessivo ocorre quando um modelo exposto a um número muito pequeno de exemplos aprende padrões que não se generalizam para novos dados, ou seja, quando o modelo começa a usar recursos irrelevantes para fazer previsões. Por exemplo, se você, como ser humano, vir apenas três imagens de pessoas que são lenhadores e três imagens de pessoas que são marinheiros e, entre elas, a única pessoa que usa um boné é um lenhador, você pode começar a pensar que usar um boné é um sinal de ser um lenhador e não um marinheiro. Assim, você seria um péssimo classificador de lenhador/marinheiro.

O ajuste excessivo é o problema central do aprendizado de máquina: se estivermos ajustando os parâmetros do nosso modelo a um determinado conjunto de dados, como podemos ter certeza de que as representações aprendidas pelo modelo serão aplicáveis a dados que ele nunca viu antes? Como você evita aprender coisas que são específicas dos dados de treinamento?

No próximo exercício, você verá maneiras de evitar o ajuste excessivo nesse modelo de classificação.

## Limpar

Antes de executar o próximo exercício, execute a seguinte célula para encerrar o kernel e liberar recursos de memória:

In [ ]:
import os, signal

os.kill(os.getpid(), signal.SIGKILL)